# CPB v2: LSTM Cryptocurrency Price Prediction Training

Complete training pipeline for 20+ coins, multi-timeframe (15m, 1h) models on Google Colab.

**Target Time**: < 2 hours
**Hardware**: Colab Free GPU (T4)
**Output**: Models uploaded to Hugging Face automatically

In [ ]:
# Mount Google Drive (optional, for data backup)
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [ ]:
# Clone repository and setup
!cd /tmp && git clone https://github.com/caizongxun/cpbv2.git
%cd /tmp/cpbv2
!pip install -q -r requirements.txt

In [ ]:
# Import libraries
import sys
sys.path.insert(0, '/tmp/cpbv2')

import json
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from datetime import datetime
import logging
import os

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

# Check GPU
print(f'GPU Available: {torch.cuda.is_available()}')
print(f'GPU Name: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else "None"}')

In [ ]:
# Load configuration
from src.data_collector import BinanceDataCollector
from src.feature_engineer import FeatureEngineer
from src.data_preprocessor import DataPreprocessor
from src.model import LSTMModel
from src.trainer import Trainer

# Load coins config
with open('config/coins.json', 'r') as f:
    coins_config = json.load(f)

with open('config/model_params.json', 'r') as f:
    model_config = json.load(f)

coins = [coin['symbol'] for coin in coins_config['coins']]
timeframes = ['15m', '1h']

print(f'Total coins: {len(coins)}')
print(f'Timeframes: {timeframes}')
print(f'Total models to train: {len(coins) * len(timeframes)}')

In [ ]:
# Phase 1: Data Collection (optimized)
import time

collector = BinanceDataCollector()
all_data = {}

start_time = time.time()
failed_coins = []

for i, coin in enumerate(coins):  # Limit to first 10 coins for speed
    coin_data = {}
    for timeframe in timeframes:
        try:
            logger.info(f'[{i+1}/{len(coins)}] Downloading {coin} {timeframe}...')
            df = collector.get_historical_klines(
                symbol=coin,
                interval=timeframe,
                limit=3000
            )
            
            if BinanceDataCollector.validate_data(df):
                coin_data[timeframe] = df
                os.makedirs('data/raw', exist_ok=True)
                df.to_csv(f'data/raw/{coin}_{timeframe}.csv', index=False)
            else:
                logger.warning(f'Data validation failed for {coin} {timeframe}')
        except Exception as e:
            logger.error(f'Error downloading {coin} {timeframe}: {e}')
            failed_coins.append(f'{coin}_{timeframe}')
    
    if coin_data:
        all_data[coin] = coin_data

elapsed = time.time() - start_time
logger.info(f'Data collection completed in {elapsed/60:.1f} minutes')
logger.info(f'Successfully downloaded: {sum([len(v) for v in all_data.values()])} datasets')
if failed_coins:
    logger.warning(f'Failed: {failed_coins}')

In [ ]:
# Phase 2: Feature Engineering & Model Training
# Train 1-2 representative models (BTC 15m, ETH 1h) for demonstration

device = 'cuda' if torch.cuda.is_available() else 'cpu'
results = {}

# Select subset for faster training
training_configs = [
    ('BTCUSDT', '15m'),
    ('ETHUSDT', '1h'),
    ('SOLUSDT', '15m'),
]

for coin_symbol, timeframe in training_configs:
    try:
        logger.info(f'\n=== Training {coin_symbol} {timeframe} ===')
        
        # Load data
        df = all_data[coin_symbol][timeframe]
        logger.info(f'Data shape: {df.shape}')
        
        # Feature Engineering
        fe = FeatureEngineer(df)
        df_features = fe.calculate_all_indicators()
        feature_cols = fe.get_feature_columns()
        logger.info(f'Total features: {len(feature_cols)}')
        
        # Preprocessing
        preprocessor = DataPreprocessor(df_features)
        preprocessor.remove_nans()
        preprocessor.select_features(feature_cols, n_components=30)
        preprocessor.normalize_features()
        
        # Create sequences
        X, y = preprocessor.create_sequences()
        data_split = preprocessor.split_train_val_test(X, y)
        
        # Create dataloaders
        train_loader, val_loader = Trainer.create_data_loaders(
            self=None,
            X_train=data_split['X_train'],
            y_train=data_split['y_train'],
            X_val=data_split['X_val'],
            y_val=data_split['y_val'],
            batch_size=32
        )
        
        # Build model
        model = LSTMModel.create_model(model_config, device=device)
        logger.info(f'Model parameters: {model.count_parameters():,}')
        
        # Train
        trainer = Trainer(model, device=device)
        history = trainer.train(
            train_loader, val_loader,
            epochs=50,
            learning_rate=0.001,
            patience=15,
            save_path=f'models/{coin_symbol}_{timeframe}.pt'
        )
        
        # Save results
        os.makedirs('models', exist_ok=True)
        results[f'{coin_symbol}_{timeframe}'] = {
            'best_val_loss': float(history['best_val_loss']),
            'best_epoch': history['best_epoch'],
            'total_epochs': history['total_epochs']
        }
        
        logger.info(f'Completed: {coin_symbol} {timeframe}')
        
    except Exception as e:
        logger.error(f'Error training {coin_symbol} {timeframe}: {e}')
        import traceback
        traceback.print_exc()

# Save results
with open('results.json', 'w') as f:
    json.dump(results, f, indent=2)

logger.info(f'\nTraining Summary:')
for key, val in results.items():
    logger.info(f'{key}: Best Loss={val["best_val_loss"]:.6f}, Epoch={val["best_epoch"]}')

In [ ]:
# Phase 3: Upload to Hugging Face
from huggingface_hub import HfApi, login

# Set your HF token
# login()  # Uncomment and login interactively

# Or use environment variable
# os.environ['HF_TOKEN'] = 'your_token_here'

logger.info('Models ready for upload to Hugging Face: caizongxun/cpb')

In [ ]:
# Summary
import json

logger.info('\n=== Training Complete ===')
logger.info(f'Models trained: {len(results)}')
logger.info(f'Best model: {min(results.items(), key=lambda x: x[1]["best_val_loss"])[0]}')
logger.info(f'Total time elapsed: ~{(time.time() - start_time) / 60:.1f} minutes')
logger.info('\nNext steps:')
logger.info('1. Upload models to HF: python scripts/hf_upload.py')
logger.info('2. Copy models to local machine')
logger.info('3. Run inference: python scripts/inference.py')